In [5]:
import tensorflow as tf

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/opt/anaconda3/envs/tf2/bin/python"
  * The NumPy version is: "1.21.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 0x0002): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /opt/anaconda3/envs/tf2/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so
  Reason: tried: '/opt/anaconda3/envs/tf2/lib/libop

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [23]:
inData1 = keras.Input(shape=(3,4),batch_size=1,dtype=tf.float32,name="left")

In [24]:
inData1

<KerasTensor: shape=(1, 3, 4) dtype=float32 (created by layer 'left')>

In [25]:
inData2 =  keras.Input(shape=(4,3),batch_size=1,dtype=tf.float32,name="right")

In [27]:
output = tf.einsum('zij,zjk->zik', inData1, inData2)

In [28]:
model = keras.Model(inputs=[inData1,inData2],outputs= output,name="einsum_test")

In [30]:
model.summary()

Model: "einsum_test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left (InputLayer)               [(1, 3, 4)]          0                                            
__________________________________________________________________________________________________
right (InputLayer)              [(1, 4, 3)]          0                                            
__________________________________________________________________________________________________
tf.einsum_4 (TFOpLambda)        (1, 3, 3)            0           left[0][0]                       
                                                                 right[0][0]                      
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [31]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [32]:
class EMoule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name);
        
    def __call__(self,x,y):
        z = tf.einsum('ij,jk->ik',x,y)
        return z

In [33]:
emodel = EMoule(name="test")

In [34]:
emodel

In [35]:
x = tf.random.uniform((2,3),-4,4,dtype=tf.float32)

In [36]:
y = tf.random.uniform((3,2),-4,4,dtype=tf.float32)

In [37]:
emodel(x,y)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-6.9396887, -5.921297 ],
       [-1.7752407, 13.653286 ]], dtype=float32)>

In [41]:
import tensorflow.compat.v1 as tf

In [43]:
from tensorflow.python.framework import graph_util

In [97]:
with tf.Session(graph=tf.Graph()) as sess:
    x = tf.placeholder(tf.float32, shape=(2,3),name="x")
    y = tf.placeholder(tf.float32, shape=(3,2),name="y")
#     b = tf.Variable(1,name='output')
    # matmul
    xy = tf.einsum('ij,jk->ik',x,y,name='res')
    
    sess.run(tf.global_variables_initializer())
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def,['res/Einsum'])
    
    with tf.gfile.FastGFile('./einsum_model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())

In [95]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_util
with tf.Session(graph=tf.Graph()) as sess:
    x = tf.placeholder(tf.float32, shape=(2,3),name="x")
    y = tf.placeholder(tf.float32, shape=(3,2),name="y")
#     b = tf.Variable(1,name='output')
    # matmul
    xy = tf.einsum('ij,jk->ik',x,y,name='res')
    print(xy)
    sess.run(tf.global_variables_initializer())
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def,['res/Einsum'])
    with tf.gfile.FastGFile('./einsum_model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())

Tensor("res/Einsum:0", shape=(2, 2), dtype=float32)


In [61]:
import torch 
from torch import nn as nn
import torch.nn.functional as F
import torch.onnx

In [82]:
class ETorchModel(nn.Module):
    def __init__(self):
        super(ETorchModel, self).__init__()
    def forward(self,x,y):
        res = torch.einsum("ij,jk->ik",x,y)
        return res

In [76]:
t_x = torch.randn(2,3,dtype=torch.float32,requires_grad=True)

In [77]:
t_x

tensor([[-1.1656, -0.3003,  0.1324],
        [-0.2542,  0.5518, -1.8208]], requires_grad=True)

In [78]:
t_y = torch.randn(3,2,dtype=torch.float32,requires_grad=True)

In [79]:
t_y

tensor([[-0.1992,  0.3544],
        [-1.5951,  0.9119],
        [-0.0811,  0.8305]], requires_grad=True)

In [80]:
ee =ETorchModel()

In [85]:
ee.eval() 

ETorchModel()

In [86]:
t_out = ee(t_x,t_y)

In [92]:
torch.onnx.export(ee,               # model being run
                  (t_x,t_y),                         # model input (or a tuple for multiple inputs)
                  "torch_einsum.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=13,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['x','y'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

ValueError: Unsupported ONNX opset version: 13

In [105]:
test_x = tf.constant(1,shape=(3,4))

In [106]:
test_x

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [157]:
test_y = tf.reshape(tf.range(12),(4,3))

In [158]:
test_y

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]], dtype=int32)>

In [163]:
tf.einsum('ij,jk',test_x,test_y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[18, 22, 26],
       [18, 22, 26],
       [18, 22, 26]], dtype=int32)>

### Create ONNX Einsum Model

In [93]:
print(torch.__version__)

1.4.0


In [134]:
tf.einsum('i...,...jk->i',test_x,test_y)

InvalidArgumentError: Output contains 1 broadcasting dimension(s) but no ellipsis (...) was found in the output subscripts. [Op:Einsum]

In [142]:
a =tf.reshape(tf.range(0,12),(3,4))

In [144]:
b = tf.reshape(tf.range(4), (4,))

In [151]:
a

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]], dtype=int32)>

In [152]:
b

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>

In [150]:
tf.einsum('i...,j->...j',a,b)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 0, 12, 24, 36],
       [ 0, 15, 30, 45],
       [ 0, 18, 36, 54],
       [ 0, 21, 42, 63]], dtype=int32)>

In [195]:
tf.einsum('ij,jk->ijk',test_x,test_y)

<tf.Tensor: shape=(3, 4, 3), dtype=int32, numpy=
array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]],

       [[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]],

       [[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]], dtype=int32)>

In [178]:
tf.einsum('ij,ik',test_x,test_y)

InvalidArgumentError: Expected dimension 3 at axis 0 of the input shaped [4,3] but got dimension 4 [Op:Einsum]

### test

In [172]:
aa = tf.reshape(tf.range(6),(3,2))

In [173]:
bb = tf.reshape(tf.range(12),(3,4))

In [175]:
tf.einsum('i...,ij->i...',aa,bb)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[  0,   6],
       [ 44,  66],
       [152, 190]], dtype=int32)>

In [1]:
import numpy as np

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/opt/anaconda3/envs/tf2/bin/python"
  * The NumPy version is: "1.21.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 0x0002): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /opt/anaconda3/envs/tf2/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so
  Reason: tried: '/opt/anaconda3/envs/tf2/lib/libop

In [4]:
tf.range(2*4*4*3)。

SyntaxError: invalid character in identifier (1228521445.py, line 1)

In [1]:
import tensorflow as tf

In [3]:
te = tf.constant([
    [
        0.030322151258587837,
        -0.033101215958595276,
        -0.042598284780979156,
        -0.07107143104076385,
        -0.0294446162879467,
        -0.06502249091863632,
        0.07586171478033066,
        0.03363031521439552
    ],
    [
        0.047669388353824615,
        -0.10604513436555862,
        -0.06247670575976372,
        -0.06360768526792526,
        0.010938183404505253,
        -0.032001424580812454,
        0.10601817816495895,
        -0.021634472534060478
    ]
])

2022-01-27 14:10:41.758433: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 14:10:41.761007: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
te

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[ 0.03032215, -0.03310122, -0.04259828, -0.07107143, -0.02944462,
        -0.06502249,  0.07586171,  0.03363032],
       [ 0.04766939, -0.10604513, -0.06247671, -0.06360769,  0.01093818,
        -0.03200142,  0.10601818, -0.02163447]], dtype=float32)>

In [7]:
tf.gather(te,[1])

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[ 0.04766939, -0.10604513, -0.06247671, -0.06360769,  0.01093818,
        -0.03200142,  0.10601818, -0.02163447]], dtype=float32)>